# <div align="Center" style="color:rgb(250, 250, 50);"> Loan Application History Query </div>

In [1]:
# Declare Library

# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')


from datetime import datetime

# %% [markdown]
## Configure Settings
# Set options or configurations as needed

# Example: pd.set_option('display.max_columns', None)



### Detailed Documentation: Loan Application History Analysis Query

#### **Objective**
This query analyzes a customer's loan application history prior to a specific reference date ("obdate") for each loan application. It calculates aggregated metrics across previous applications to understand customer behavior patterns.

---

### **Query Structure**
The query uses two CTEs:
1. **`a1`**: Enriches loan data with status information
2. **`b1`**: Computes historical metrics based on previous applications

---

### **CTE a1: Loan Status Enrichment**
```sql
a1 as (
  select 
    lmt.customerId,
    lmt.digitalLoanAccountId,
    ... -- loan attributes
    case 
      when lmt.disbursementDateTime is not null and e1.loanStatus is null then '4. Active Loan' 
      when lmt.disbursementDateTime is not null and lower(e1.loanStatus) like '%sold%' then '1. Sold'
      when lmt.disbursementDateTime is not null and lower(e1.loanStatus) like '%settl%' then '2. Settled'
      when lmt.disbursementDateTime is not null and lower(e1.loanStatus) like '%compl%' then '3. Completed'
      else '5. No Disbursed loan' 
    end loanStatus,
    e1.eventdate
  from loan_master_table lmt
  left join (
    select loanAccountNumber, loanStatus, min(bucketDate) eventdate 
    from loan_bucket_flow_report_core 
    where loanStatus in ('Settled', 'Completed', 'Sold')
    group by 1, 2
  ) e1 on e1.loanAccountNumber = lmt.loanAccountNumber
)
```

#### **Key Logic**
1. **Status Classification**:
   - `Active Loan`: Disbursed but no status in flow report
   - `Sold`: Disbursed and status contains "sold"
   - `Settled`: Disbursed and status contains "settl"
   - `Completed`: Disbursed and status contains "compl"
   - `No Disbursed loan`: Not disbursed

2. **Subquery `e1`**:
   - Finds earliest event date for final loan statuses
   - Aggregates by loan account number and status

---

### **CTE b1: Historical Metrics Calculation**
```sql
b1 as (
  select 
    lmt.customerId,
    lmt.digitalLoanAccountId,
    case  -- Define obdate
      when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
      when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null then coalesce(lmt.startApplyDateTime, lmt.startInitiateDateTime)
      when lmt.new_loan_type not like 'Flex-up' then coalesce(lmt.termsAndConditionsSubmitDateTime, lmt.startApplyDateTime, lmt.startInitiateDateTime)
    end obdate,
    ... -- 40+ aggregated metrics
  from loan_master_table lmt 
  left join a1 
    on a1.customerId = lmt.customerId 
    and [a1_derived_date] < [lmt_derived_date] -- Date comparison logic
  group by 1,2,3
)
```

#### **Core Components**
1. **`obdate` (Reference Date)**:
   - `Flex-up` loans: Disbursement date > Apply date > Initiate date
   - Other loans: T&C submission > Apply date > Initiate date

2. **Historical Metrics**:
   - **Counts**:
     - Total loans tried before `obdate`
     - Approved/rejected/disbursed counts
     - Loan type breakdowns (Quick, SIL, Other)
     - Status-based counts (Sold/Settled/Completed/Active)
   - **Financial Metrics**:
     - Min/Max requested/approved/disbursed amounts
     - Min/Max requested/approved/disbursed tenures

3. **Date Comparison Logic**:
   - Compares derived dates using same rules as `obdate`:
     ```sql
     case when a1.new_loan_type ='Flex-up' then coalesce(a1.disbursementDateTime, a1.startApplyDateTime, a1.startInitiateDateTime)
          else coalesce(a1.termsAndConditionsSubmitDateTime, a1.startApplyDateTime, a1.startInitiateDateTime)
     end < obdate
     ```

---

### **Key Metrics Summary**
| Category | Metrics | Description |
|----------|---------|-------------|
| **Volume Metrics** | `cnt_loans_tried_bf_obdate` | Total previous applications |
|  | `cnt_loans_rejected_bf_obdate` | Previously rejected loans |
|  | `cnt_quick_loan_tried_bf_obdate` | Quick loan applications |
| **Financial Metrics** | `min_loan_amt_requested_bf_obdate` | Smallest requested amount |
|  | `max_loan_amt_approved_bf_obdate` | Largest approved amount |
|  | `min_loan_tenure_disb_bf_obdate` | Shortest disbursed tenure |
| **Status Metrics** | `no_of_loans_sold_bf_obdate` | Loans sold to third parties |
|  | `no_of_loans_active_bf_obdate` | Currently active loans |

---

### **Technical Notes**
1. **Null Handling**:
   - `coalesce()` equivalent logic in `case` statements
   - Zero-default in financial aggregates (`else 0`)

2. **Performance**:
   - Self-join on `customerId` may be expensive
   - Repeated date derivation logic could be optimized

3. **Edge Cases**:
   - "Other loans" defined as neither Quick nor SIL
   - Flex-up loans prioritize disbursement date

---

### **Example Output**
| customerId | digitalLoanAccountId | obdate | cnt_loans_tried_bf_obdate | max_loan_amt_approved... |
|------------|----------------------|--------|---------------------------|--------------------------|
| 12345 | DL-2023-001 | 2023-01-15 | 2 | 5000 |
| 12345 | DL-2023-002 | 2023-03-22 | 3 | 7500 |

---

### **Usage Scenarios**
1. Customer risk profiling
2. Loan approval decision support
3. Customer lifetime value analysis
4. Product preference analysis (Quick vs SIL vs Other)

This query provides a comprehensive view of a customer's loan application history prior to each new application, enabling data-driven decision making in lending operations.

## Query

In [2]:
sq = """ 
with a1 as 
(select 
  lmt.customerId,
  lmt.digitalLoanAccountId,
  lmt.startApplyDateTime,
  lmt.startInitiateDateTime,
  lmt.termsAndConditionsSubmitDateTime,
  lmt.applicationStatus,
  lmt.new_loan_type,
  lmt.flagApproval,
  lmt.flagRejection,
  lmt.flagDisbursement,
  lmt.disbursementDateTime,
  lmt.loanRequestAmount,
  lmt.loanRequestTenure,
  lmt.approvedLoanAmount,
  lmt.approvedLoanTenure,
  lmt.disbursedLoanAmount,
  lmt.disbursedLoanTenur,
  case when lmt.disbursementDateTime is not null and e1.loanStatus is null then '4. Active Loan' 
       when lmt.disbursementDateTime is not null and e1.loanStatus is not null and lower(e1.loanStatus) like '%sold%'  then '1. Sold'
       when lmt.disbursementDateTime is not null and e1.loanStatus is not null and lower(e1.loanStatus) like '%settl%'  then '2. Settled'
       when lmt.disbursementDateTime is not null and e1.loanStatus is not null and lower(e1.loanStatus) like '%compl%'  then '3. Completed'
       else '5. No Disbursed loan' end loanStatus,
  e1.eventdate
from `risk_credit_mis.loan_master_table` lmt
left join (select loanAccountNumber, loanStatus, min(bucketDate) eventdate from `risk_credit_mis.loan_bucket_flow_report_core` where loanStatus in ('Settled', 'Completed', 'Sold')
group by 1, 2) e1 on e1.loanAccountNumber = lmt.loanAccountNumber
),
b1 as 
(select 
lmt.customerId,
lmt.digitalLoanAccountId,
case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end obdate,
min(lmt.new_loan_type) new_loan_type,
min(lmt.applicationStatus) application_status,
count(distinct a1.digitalLoanAccountId) cnt_loans_tried_bf_obdate, 
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and a1.flagRejection = 1 then  a1.digitalLoanAccountId end) cnt_loans_rejected_bf_obdate, 
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and a1.flagApproval = 1 then  a1.digitalLoanAccountId end) cnt_loans_approved_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and a1.flagDisbursement = 1 then  a1.digitalLoanAccountId end) cnt_loans_disbursed_bfobdate,  
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and lower(a1.new_loan_type) like '%quick%' then a1.digitalLoanAccountId end) cnt_quick_loan_tried_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and lower(a1.new_loan_type) like '%quick%' and a1.flagRejection = 1 then a1.digitalLoanAccountId end) cnt_quick_loan_rejected_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and lower(a1.new_loan_type) like '%quick%' and a1.flagApproval = 1 then a1.digitalLoanAccountId end) cnt_quick_loan_approved_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and lower(a1.new_loan_type) like '%quick%' and a1.flagDisbursement = 1 then a1.digitalLoanAccountId end) cnt_quick_loan_disb_bf_obdate,

count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and lower(a1.new_loan_type) like '%sil%' then a1.digitalLoanAccountId end) cnt_sil_loan_tried_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and lower(a1.new_loan_type) like '%sil%' and a1.flagRejection = 1 then a1.digitalLoanAccountId end) cnt_sil_loan_rejected_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and lower(a1.new_loan_type) like '%sil%' and a1.flagApproval = 1 then a1.digitalLoanAccountId end) cnt_sil_loan_approved_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and lower(a1.new_loan_type) like '%sil%' and a1.flagDisbursement = 1 then a1.digitalLoanAccountId end) cnt_sil_loan_disb_bf_obdate,

count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and (lower(a1.new_loan_type) not like '%sil%' or lower(a1.new_loan_type) not like '%quick%') then a1.digitalLoanAccountId end) cnt_other_loan_tried_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and (lower(a1.new_loan_type) not like '%sil%' or lower(a1.new_loan_type) not like '%quick%') and a1.flagRejection = 1 then a1.digitalLoanAccountId end) cnt_other_loan_rejected_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and (lower(a1.new_loan_type) not like '%sil%' or lower(a1.new_loan_type) not like '%quick%') and a1.flagApproval = 1 then a1.digitalLoanAccountId end) cnt_other_loan_approved_bf_obdate,
count(distinct case when a1.startApplyDateTime < lmt.startApplyDateTime and (lower(a1.new_loan_type) not like '%sil%' or lower(a1.new_loan_type) not like '%quick%') and a1.flagDisbursement = 1 then a1.digitalLoanAccountId end) cnt_other_loan_disb_bf_obdate,


min( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.loanRequestAmount else 0 end) min_loan_amt_requested_bf_obdate,
max( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.loanRequestAmount else 0 end) max_loan_amt_requested_bf_obdate,

min( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.loanRequestTenure else 0 end) min_loan_tenure_requested_bf_obdate,
max( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.loanRequestTenure else 0 end) max_loan_tenure_requested_bf_obdate,

min( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.approvedLoanAmount else 0 end) min_loan_amt_approved_bf_obdate,
max( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.approvedLoanAmount else 0 end) max_loan_amt_approved_bf_obdate,

min( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.approvedLoanTenure else 0 end) min_loan_tenure_approved_bf_obdate,
max( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.approvedLoanTenure else 0 end) max_loan_tenure_approved_bf_obdate,

min( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.disbursedLoanAmount else 0 end) min_loan_amt_disb_bf_obdate,
max( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.disbursedLoanAmount else 0 end) max_loan_amt_disb_bf_obdate,

min( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.disbursedLoanTenur else 0 end) min_loan_tenure_disb_bf_obdate,
max( case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) then a1.disbursedLoanTenur else 0 end) max_loan_tenure_disb_bf_obdate,

count(distinct case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) and loanStatus like '1. Sold' then a1.digitalLoanAccountId end) no_of_loans_sold_bf_obdate,
count(distinct case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) and loanStatus like '2. Settled' then a1.digitalLoanAccountId end) no_of_loans_settled_bf_obdate,
count(distinct case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) and loanStatus like '3. Completed' then a1.digitalLoanAccountId end) no_of_loans_completed_bf_obdate,
count(distinct case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) <(case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) and loanStatus like '4. Active Loan' then a1.digitalLoanAccountId end) no_of_loans_active_bf_obdate,
count(distinct case when (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end) < (case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end) and loanStatus like '5. No Disbursed loan'  then a1.digitalLoanAccountId end) customer_did_not_have_disb_loans,


from `risk_credit_mis.loan_master_table` lmt 
left join a1 on a1.customerId = lmt.customerId 
and (case when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is not null then a1.disbursementDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type ='Flex-up' and a1.disbursementDateTime is null and a1.startApplyDateTime is null then a1.startInitiateDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is not null then a1.termsAndConditionsSubmitDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is not null then a1.startApplyDateTime
     when a1.new_loan_type not like 'Flex-up' and a1.termsAndConditionsSubmitDateTime is null and a1.startApplyDateTime is  null then a1.startInitiateDateTime
     end)  < 
(case when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is not null then lmt.disbursementDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type ='Flex-up' and lmt.disbursementDateTime is null and lmt.startApplyDateTime is null then lmt.startInitiateDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is not null then lmt.termsAndConditionsSubmitDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is not null then lmt.startApplyDateTime
     when lmt.new_loan_type not like 'Flex-up' and lmt.termsAndConditionsSubmitDateTime is null and lmt.startApplyDateTime is  null then lmt.startInitiateDateTime
     end)
group by 1,2,3
)
select * from b1
--  where customerId = 2836845
;
"""
df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 1be3badb-8b48-4a8f-8007-c1d79fb240d2 successfully executed: 100%|██████████|

C:\Users\Dwaipayan\AppData\Roaming\Python\Python312\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [3]:
print(f"The query returned {len(df)} rows.")

The query returned 2596627 rows.


In [4]:
df.columns.to_list()

['customerId',
 'digitalLoanAccountId',
 'obdate',
 'new_loan_type',
 'application_status',
 'cnt_loans_tried_bf_obdate',
 'cnt_loans_rejected_bf_obdate',
 'cnt_loans_approved_bf_obdate',
 'cnt_loans_disbursed_bfobdate',
 'cnt_quick_loan_tried_bf_obdate',
 'cnt_quick_loan_rejected_bf_obdate',
 'cnt_quick_loan_approved_bf_obdate',
 'cnt_quick_loan_disb_bf_obdate',
 'cnt_sil_loan_tried_bf_obdate',
 'cnt_sil_loan_rejected_bf_obdate',
 'cnt_sil_loan_approved_bf_obdate',
 'cnt_sil_loan_disb_bf_obdate',
 'cnt_other_loan_tried_bf_obdate',
 'cnt_other_loan_rejected_bf_obdate',
 'cnt_other_loan_approved_bf_obdate',
 'cnt_other_loan_disb_bf_obdate',
 'min_loan_amt_requested_bf_obdate',
 'max_loan_amt_requested_bf_obdate',
 'min_loan_tenure_requested_bf_obdate',
 'max_loan_tenure_requested_bf_obdate',
 'min_loan_amt_approved_bf_obdate',
 'max_loan_amt_approved_bf_obdate',
 'min_loan_tenure_approved_bf_obdate',
 'max_loan_tenure_approved_bf_obdate',
 'min_loan_amt_disb_bf_obdate',
 'max_loan_amt_d

In [5]:
# Save locally
local_path = "temp_data.parquet"
df.to_parquet(local_path, engine='pyarrow')

In [6]:
from google.cloud import storage

def upload_to_gcs(bucket_name, source_file, destination_blob_name):
    """Uploads a file to GCS bucket"""
    storage_client = storage.Client(project="prj-prod-dataplatform")
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file)

# GCS Configuration
CURRENT_DATE = datetime.now().strftime("%Y-%m-%d")
BUCKET_NAME = "prod-asia-southeast1-tonik-aiml-workspace"
DESTINATION_PATH = f"DC/Feature_Mart/Data/{CURRENT_DATE}_Loan_Application_history.parquet"  # Path within bucket

# Upload
upload_to_gcs(BUCKET_NAME, local_path, DESTINATION_PATH)
print(f"File uploaded to gs://{BUCKET_NAME}/{DESTINATION_PATH}")

File uploaded to gs://prod-asia-southeast1-tonik-aiml-workspace/DC/Feature_Mart/Data/2025-07-22_Loan_Application_history.parquet
